In [ ]:
import osmnx as ox, geopandas as gpd
import srtm as sr
import networkx as nx
%matplotlib inline
ox.config(log_file=True, log_console=True, use_cache=True)
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from haversine import haversine
import math
from shapely.geometry import Point, LineString

In [ ]:
# PARAMETERS OF THE VEHICLE TO SIMULATE

#Area frontale del veicolo
A = 2.27
# Massa del veicolo
Mcar = 1580
# Massa del guidatore
Md = 90
# Coefficiente di resistenza aerodinamica
Cd = 0.29
# Coefficiente di resistenza al rotolamento
R = 0.012
# Capacità della batteria
eBatCap = 38
# Massa totale
TotalMass = Mcar+Md
# Efficienza della trasmissione
effGear = 0.97
# Efficienza del motore elettrico
effMot = 0.95
# Potenza ausiliaria
Paux = 0
# Efficienza della batteria
effBattCharge = 0.95
# Efficienza scaricamento batteria
effBattDischarge = 0.98
# Accelerazione di gravità
g = 9.81
# Densità aria
rho = 1.2041
# Regeneration ratio (G = 0.35 in ECO MODE)
regenRatio = 0.35
def evBatteryConsumption (distanceFromPrev, timeGap, speed, accel, alpha):
    # Resistenza al rotolamento
    Frr = R * (TotalMass) * g * math.cos(alpha)
    # Resistenza areodinamica
    Fa = 0.5 * A *Cd * rho * math.pow(speed, 2)
    # Gravità
    Fgo = (TotalMass) * g * math.sin(alpha)
    # Forza d'inerzia
    Fi = 1.05 * (TotalMass) * accel
    # Forza totale
    Ftot = Frr + Fa + Fgo + Fi
    # Forza trazione meccanica
    Ptot = Ftot * speed
    
    PmotOut = 0
    if (Ptot >= 0):
        PmotOut = Ptot/effGear
    else:
        PmotOut = regenRatio * Ptot * effGear
    
    PmotIn = 0
    if (PmotOut >= 0):
        PmotIn = PmotOut/effMot
    else:
        PmotIn = PmotOut*effMot
    
    Pbat = PmotIn + Paux
    
    # Modellazione batteria
    eBat = 0
    if(Pbat >= 0):
        eBat = Pbat * timeGap/effBattDischarge
    else:
        eBat = Pbat * timeGap * effBattDischarge
    
    # Calcolo DeltaSoC
    kWh2Ws = 3600*1e3
    deltaSoC = eBat/(eBatCap*kWh2Ws)
    
    return deltaSoC

### Carica il grafo da disco

In [ ]:
def caricaMappaDaDisco (fileName,folder):
    G = ox.save_load.load_graphml(fileName, folder=folder)
    return G

In [ ]:
ToscanaTorrette = 'toscana_torrette_maxspeed.graphml'

G = caricaMappaDaDisco(ToscanaTorrette,None)

In [ ]:
speedList = []
conta  = 0
for i in G.edges:
    conta+=1
    if G.edges[i]['maxspeed'] not in speedList:
        speedList.append(G.edges[i]['maxspeed'])
print("Numero totale di archi: {}".format(conta))

In [ ]:
colori = ox.plot.get_colors(len(speedList), cmap='RdYlGn', start=0.3, stop=1.0, alpha=1.0, return_hex=True)

In [ ]:
len(speedList)

In [ ]:
nc = ['blue' if 'tipo' in data else 'w' for n , data in G.nodes(data=True)]
ne = ['r' if 'tipo' in data else 'w' for n , data in G.nodes(data=True)]
ns = [50 if 'tipo' in data else 0 for n , data in G.nodes(data=True)]
ec = [colori[0] if data['maxspeed'] == speedList[0] else colori[1] if data['maxspeed'] == speedList[1] else colori[2] if data['maxspeed'] == speedList[2] else colori[3] if data['maxspeed'] == speedList[3] else colori[4] if data['maxspeed'] == speedList[4] else colori[5] if data['maxspeed'] == speedList[5] else colori[6] if data['maxspeed'] == speedList[6] else colori[7] if data['maxspeed'] == speedList[7] else colori[8] if data['maxspeed'] == speedList[8] else 'w' for u, v, key, data in G.edges(keys=True, data=True)]
fig, ax = ox.plot_graph(G, fig_height=30, bgcolor = 'w', show = True, save = True,
                        filename='plot toscana archi in base a maxspeed nodi charge rossi', 
                        dpi=80, node_color = nc, node_size = ns, node_edgecolor = ne,node_zorder=2,edge_color=ec)

### Aggiunge il valore di elevation ai nodi

In [ ]:
elevation_data = sr.get_data()
#Aggiunge tag 'elev'= (elevation del nodo) ad ogni nodo
for i in G.nodes:
    ln = G.nodes[i]['y']
    la = G.nodes[i]['x']
    tempelev = elevation_data.get_elevation(ln,la)
    G.add_node(i, elev = tempelev)


#Sostituisce i valori None di elevation con l'elevation media dei nodi vicini
for j in G.nodes:
    if G.nodes[j]['elev'] is None:
        closelev = G.neighbors(j)
        conta = 0
        tempelev = 0
        for i in closelev:
            if i != j:
                tempelev = tempelev + G.nodes[i]['elev']
                conta += 1
        G.add_node(j, elev = tempelev/conta)
        print('a', G.nodes[j])

#Controlla se ci sono nodi con elevation = None
nulli  = 0
for i in G.nodes:
    if G.node[i]['elev'] is None:
        nulli+=1
print("I nodi con valore di elevation nullo sono {}".format(nulli))

In [ ]:
#Effettua il plot del grafo colorando i nodi in base all'elevation
nc = ox.get_node_colors_by_attr(G, 'elev', cmap='inferno', num_bins=20)
fig, ax = ox.plot_graph(G, fig_height=30, bgcolor = 'black',node_color=nc, node_size=35, node_zorder=2, edge_color='#dddddd')

### Calcola il dislivello tra un nodo e il suo vicino

In [ ]:
for u in G.nodes:
    for v in G.neighbors(u):
        dislivello = (G.node[v]['elev'])-(G.node[u]['elev'])
#         print(dislivello)
        G.add_edge(u,v,key = 0, slope = float(dislivello))

In [ ]:
for u,v,k in G.edges:
    if 'slope' not in G[u][v][k]:
        G.add_edge(u,v,key = k, slope = 0)

In [ ]:
for u,v,k in G.edges:
    if abs(G[u][v][k]['slope'])>G[u][v][k]['length']:
        G.add_edge(u,v,key = k, slope = 0)    

In [ ]:
pendenze.sort()

In [ ]:
ec = ['g' if data['slope'] < 0 else 'b' if data['slope'] == 0 else 'r' for u, v, key, data in G.edges(keys=True, data=True)]
fig, ax = ox.plot_graph(G, fig_height=30,show = True, save = True,
                        filename='plot toscana archi in base alla pendenza', 
                        dpi=80, bgcolor = 'w',node_color='black', node_size=0, edge_color=ec)

In [ ]:
pendenze =[]
pendenzeTra10e20 = 0
pendenzeSopra20 = 0
for u,v,k in G.edges:
    if G[u][v][k]['slope'] != 0:
        dislivello = G[u][v][k]['slope']
        lunghezzaStrada = G[u][v][k]['length']
        distanza = math.sqrt((lunghezzaStrada*lunghezzaStrada)-(dislivello*dislivello))
        percDiscesa = dislivello/distanza
        if percDiscesa >= -0.25 and percDiscesa <= 0.25:
            pendenze.append(percDiscesa*100)
        elif percDiscesa > 20:
            pendenzeSopra20 +=1
        angDiscesa = math.atan(percDiscesa)
        G.add_edge(u,v,key = k, slopePerc = float(percDiscesa), slopeAngle = float(angDiscesa))
    else:
        G.add_edge(u,v,key = k, slopePerc = 0, slopeAngle = 0)
print("ci sono {} pendenze tra il 10 e il 20 % e {} pendenze superiori al 20 %".format(pendenzeTra10e20,pendenzeSopra20))

In [ ]:
pendenze

In [ ]:
sns.distplot(pendenze, hist=True, kde=True, bins = 'auto',
             color = 'black', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2})
plt.xlabel('Pendenza%')
plt.ylabel('Frequency')

In [ ]:
# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=pendenze, bins='auto', color='#0504aa',density = True,
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Velocità (km/h)')
plt.ylabel('Frequency')
# plt.title('Distribuzione Maxspeed tra in valori non nulli (10%)')
maxfreq = n.max()
# Set a clean upper y-axis limit.
# plt.ylim(top = np.ceil(maxfreq))

In [ ]:
for u,v,k in G.edges:
    tempoDiPercorrenza = G[u][v][k]['length']/(int(G[u][v][k]['maxspeed'])/3.6)
    G.add_edge(u,v,key = k, traveltime = float(tempoDiPercorrenza) )
    

In [ ]:
for u,v,k in G.edges:
    consumo = evBatteryConsumption (G[u][v][k]['length'], G[u][v][k]['traveltime'], int(G[u][v][k]['maxspeed'])/3.6, 0, G[u][v][k]['slopeAngle'])
    G.add_edge(u,v,key = k, consumption = float(consumo*(eBatCap*(3600*1e3))))

In [ ]:
for u,v,k in G.edges:
    G.add_edge(u,v,key = k, maxspeed = int(G[u][v][k]['maxspeed']))

In [ ]:
# for i in G.nodes():
#     if 'tipo' in G.nodes[i]:
#         G.add_edge(i,i,0, traveltime = None, consumption = None)

In [ ]:
def salvaMappa (Graph,filename,folder,gephi):
    ox.save_load.save_graphml(Graph, filename=filename, folder=folder, gephi=gephi)

In [ ]:
#Salva il grafo su disco
# filename = 'toscanatorrette.graphml'
filename = 'toscana_torrette_completo.graphml'


salvaMappa(G,filename,None,None)